# Common Transforms
## Summary of common transforms

*Filter*
- Filter all elements in a **PCollection** with `beam.Filter(<predicate-fn>)`
    - `<predicate-fn>` can be pre-defined or lambda form
    - Passing extra args to `beam.Filter()` passes those args into the second arg onwards of `<predicate-fn>`
        - Using this info, you can pass a *singleton* or a dictionary into the second arg, and use that to filter your **PCollection**

*Aggregations - Count*


## Filter

`PCollection`s can be filtered using the `Filter` transform. Create a filter by supplying a predicate, and `Filter` will remove all elements of the `PCollection` that don't satisfy the predicate.

e.g.:

In [4]:
import apache_beam as beam

def is_perennial(plant):
  return plant['duration'] == 'perennial'

with beam.Pipeline() as p:
    perennials = (
        p 
        | 'Gardening plants' >> beam.Create([
            {'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'},
            {'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'},
            {'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'},
            {'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'},
            {'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'},
        ])
        # | 'Filter perennials' >> beam.Filter(is_perennial)
        # Alternatively:
        | 'Filter perennials' >> beam.Filter(lambda plant: plant['duration'] == 'perennial')
        | beam.Map(print)
    )

{'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'}
{'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'}
{'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'}


### Filtering with multiple arguments
You can also pass multiple arguments to `Filter`, they are passed as additional positional arguments or keyword arguments to the function.

In [5]:
def has_duration(plant, duration):
    return plant['duration'] == duration

with beam.Pipeline() as p:
    perennials = (
        p
        | 'Gardening plants' >> beam.Create([
            {'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'},
            {'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'},
            {'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'},
            {'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'},
            {'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'},
        ])
        | 'Filter perennials' >> beam.Filter(has_duration, 'perennial') # First arg is filled in with the piped in PCollection
        | beam.Map(print)
    )

{'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'}
{'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'}
{'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'}


### Filtering with side inputs as singletons

A *singleton* is a `PCollection` with only one element. We can use these to filter our input `PCollection`.

In [6]:
with beam.Pipeline() as p:

    perennial = p | 'Perennial' >> beam.Create(['perennial'])

    perennials = (
        p 
        | 'Gardening plants' >> beam.Create([
            {'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'},
            {'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'},
            {'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'},
            {'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'},
            {'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'},
        ])
        | 'Filter perennials' >> beam.Filter(
            lambda plant, duration: plant['duration'] == duration,
            duration = beam.pvalue.AsSingleton(perennial),
        )
        | beam.Map(print)
    )

{'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'}
{'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'}
{'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'}


### Filtering with side inputs as iterators

If a `PCollection` has multiple values, you can pass that in as an iterator. This will access the elements lazily as needed, so can iterate over `PCollection`s that don't fit in memory.

In [8]:
with beam.Pipeline() as p:

    valid_durations = p | 'Valid durations' >> beam.Create([
        'annual',
        'biennial',
        'perennial',
    ])

    perennials = (
        p 
        | 'Gardening plants' >> beam.Create([
            {'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'},
            {'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'},
            {'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'},
            {'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'},
            {'icon': '🥔', 'name': 'Potato', 'duration': 'PERENNIAL'}, # note this is in all caps now
        ])
        | 'Filter perennials' >> beam.Filter(
            lambda plant, valid_durations: plant['duration'] in valid_durations,
            valid_durations = beam.pvalue.AsIter(valid_durations),
        )
        | beam.Map(print)
    )

{'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'}
{'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'}
{'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'}
{'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'}


### Filtering with side inputs as dictionaries

If a `PCollection` is small enough to fit in memory, it can be passed as a dictionary into `Filter`. Each element must be a `(key, value)` pair. If the elements won't fit in memory, use `AsIter()` instead.

In [9]:
with beam.Pipeline() as p:

    keep_duration = p | 'Duration filters' >> beam.Create([
        ('annual', False),
        ('biennial', False),
        ('perennial', True),
    ])

    perennials = (
        p 
        | 'Gardening plants' >> beam.Create([
            {'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'},
            {'icon': '🥕', 'name': 'Carrot', 'duration': 'biennial'},
            {'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'},
            {'icon': '🍅', 'name': 'Tomato', 'duration': 'annual'},
            {'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'}, # note this is in all caps now
        ])
        | 'Filter perennials' >> beam.Filter(
            lambda plant, keep_duration: keep_duration[plant['duration']],
            keep_duration = beam.pvalue.AsDict(keep_duration),
        )
        | beam.Map(print)
    )

{'icon': '🍓', 'name': 'Strawberry', 'duration': 'perennial'}
{'icon': '🍆', 'name': 'Eggplant', 'duration': 'perennial'}
{'icon': '🥔', 'name': 'Potato', 'duration': 'perennial'}


### Chaining `Filter` statements

We can also chain `Filter` statements.

In [10]:
# Output PCollection
class Output(beam.PTransform):
    class _OutputFn(beam.DoFn):
        def __init__(self, prefix=''):
            super().__init__()
            self.prefix = prefix

        def process(self, element):
            print(self.prefix+str(element))

    def __init__(self, label=None,prefix=''):
        super().__init__(label)
        self.prefix = prefix

    def expand(self, input):
        input | beam.ParDo(self._OutputFn(self.prefix))

with beam.Pipeline() as p:

    string = "To be, or not to be: that is the question: Whether 'tis nobler in the mind to suffer The slings and arrows of outrageous fortune, Or to take arms against a sea of troubles, And by opposing end them. To die: to sleep"
  
    (
        p  
        | 'Convert to words' >> 
            beam.Create(string.split(' '))
        | 'Filter for words starting with a' >>
            beam.Filter(lambda word: word[0] == 'a')
        | 'Filter for words that have more than three chars' >>
            beam.Filter(lambda word: len(word) > 3)          
        | 'Output' >>
            Output(prefix='PCollection filtered value: ')                    
    )

PCollection filtered value: arrows
PCollection filtered value: arms
PCollection filtered value: against


## Aggregations
### Count

`Count` provides transformations for counting elements; either globally within the `PCollection`, or by each key.

#### Count all elements

Use `Count.Globally()` to count all elements in a `PCollection`, even duplicated elements.

In [11]:
with beam.Pipeline() as p:
    total_elements = (
        p 
        | 'Create plants' >> beam.Create(['🍓', '🥕', '🥕', '🥕', '🍆', '🍆', '🍅', '🍅', '🍅', '🌽'])
        | 'Count all elements' >> beam.combiners.Count.Globally()
        | beam.Map(print)
    )

10


#### Counting per key

Use `Count.PerKey()` to count the elements for each unique key in a `PCollection` of key-values.

In [12]:
with beam.Pipeline() as p:
    total_elements_per_keys = (
        p 
        | 'Create plants' >> beam.Create([
            ('spring', '🍓'),
            ('spring', '🥕'),
            ('summer', '🥕'),
            ('fall', '🥕'),
            ('spring', '🍆'),
            ('winter', '🍆'),
            ('spring', '🍅'),
            ('summer', '🍅'),
            ('fall', '🍅'),
            ('summer', '🌽'),
        ])
        | 'Count elements per key' >> beam.combiners.Count.PerKey()
        | beam.Map(print)
    )

('spring', 4)
('summer', 3)
('fall', 2)
('winter', 1)


In [16]:
# Counts per key and ignores value
with beam.Pipeline() as p:
    (
        p 
        | beam.Create([(1, 36), (2, 91), (3, 33), (3, 11), (4, 67),])
        | beam.combiners.Count.PerKey() 
        | beam.Map(print)
    )

# Counts elements in PCollection, which is 5 tuples
with beam.Pipeline() as p:
    (
        p 
        | beam.Create([(1, 36), (2, 91), (3, 33), (3, 11), (4, 67),])
        | beam.combiners.Count.Globally() 
        | beam.Map(print)
    )

(1, 1)
(2, 1)
(3, 2)
(4, 1)
5


#### Counting unique elements

Use `Count.PerElement()` to count only the unique elements in a `PCollection`.

In [13]:
with beam.Pipeline() as p:
    total_unique_elements = (
        p 
        | 'Create produce' >> beam.Create(['🍓', '🥕', '🥕', '🥕', '🍆', '🍆', '🍅', '🍅', '🍅', '🌽'])
        | 'Count unique elements' >> beam.combiners.Count.PerElement()
        | beam.Map(print)
    )

('🍓', 1)
('🥕', 3)
('🍆', 2)
('🍅', 3)
('🌽', 1)


#### Counting strings

In [19]:
class SplitWords(beam.DoFn):
    def __init__(self, delimiter=' '):
        self.delimiter = delimiter

    def process(self, text):
        for word in text.split(self.delimiter):
            yield word

with beam.Pipeline() as p:
    (
        p 
        | beam.Create(["To be, or not to be: that is the question: Whether 'tis nobler in the mind to suffer, the slings and arrows of outrageous fortune, or to take arms against a sea of troubles, and by opposing end them. To die: to sleep"])
        | 'Split sentence into words' >> 
            beam.ParDo(SplitWords()) 
        | 'Count number of times each word appears' >> 
            beam.combiners.Count.PerElement()
        | 'Filter for those that appear > 1 times' >> 
            beam.Filter(lambda x: x[1] > 1)
        | Output(prefix='PCollection filtered value: ')
    )

PCollection filtered value: ('To', 2)
PCollection filtered value: ('or', 2)
PCollection filtered value: ('to', 4)
PCollection filtered value: ('the', 3)
PCollection filtered value: ('and', 2)
PCollection filtered value: ('of', 2)


### Sum
#### Sum of all elements in `PCollection`

Find the global sum of a `PCollection` by running `CombineGlobally(sum)`.

In [20]:
with beam.Pipeline() as p:
  total = (
    p 
    | 'Create numbers' >> beam.Create([3, 4, 1, 2])
    | 'Sum values' >> beam.CombineGlobally(sum)
    | beam.Map(print)
  )

10


In [22]:
with beam.Pipeline() as p:
  total = (
    p 
    | 'Create numbers' >> beam.Create([(3, 4), (1, 2)])
    # Need to flatten if elements in PCollection not already flat
    # otherwise run into error
    | 'Flatten' >> beam.FlatMap(lambda x: x)
    | 'Sum values' >> beam.CombineGlobally(sum)
    | beam.Map(print)
  )

10


#### Sum of elements by each key

Use `Combine.PerKey()` to get the sum of all values grouped by each unique key in the `PCollection`'s key-values.

In [23]:
with beam.Pipeline() as p:
    totals_per_key = (
        p 
        | 'Create produce' >> beam.Create([
            ('🥕', 3),
            ('🥕', 2),
            ('🍆', 1),
            ('🍅', 4),
            ('🍅', 5),
            ('🍅', 3),
        ])
        | 'Sum values per key' >> beam.CombinePerKey(sum)
        | beam.Map(print)
    )

('🥕', 5)
('🍆', 1)
('🍅', 12)
